In [1]:
!git clone https://github.com/hadoop-itmo/sketches-fressbish.git

Cloning into 'sketches-fressbish'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 17 (delta 2), reused 2 (delta 2), pack-reused 11 (from 1)
Receiving objects: 100% (17/17), 21.03 KiB | 358.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [3]:
pip install mmh3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.9 MB/s eta 0:00:00


In [8]:
import mmh3
import math
import numpy as np
import pandas as pd
from sketches_fressbish import utils

# Задание 1

Реализуем Bloom-Filter на одной хеш-функции.

In [9]:
class numpyBloomFilter:
    def __init__(self, n):
        self.n = n
        self.bit_array = np.zeros(self.n, dtype=bool)  # битовый массив numpy

    def put(self, s):
        hash_value = mmh3.hash(s) % self.n # хэш
        self.bit_array[hash_value] = True

    def get(self, s):
        hash_value = mmh3.hash(s) % self.n # хэш
        return self.bit_array[hash_value]

    def size(self):
        return np.sum(self.bit_array) # кол-во единиц в массиве

### Тестируем

In [10]:
bf_sizes = [8, 64, 1024, 65536, 16777216]
set_sizes = [5, 50, 500, 5000, 5000000]

In [13]:
for set_size in set_sizes:
    utils.gen_uniq_seq(name = str(set_size),
                n_records = set_size)

0
0
0
0
0
1000000
2000000
3000000
4000000


In [14]:
result = []

for bf_size in bf_sizes:
    for set_size in set_sizes:
        bf_int = numpyBloomFilter(bf_size)
        fp_count = 0
        with open(f'{set_size}') as file:
            for line in file:
                if bf_int.get(line):
                    fp_count += 1
                bf_int.put(line)
            ones_count_int = bf_int.size()

        result.append(
            {
                'bf_size' : bf_size,
                'set_size' : set_size,
                'fp_count' : fp_count,
                'ones_count' : ones_count_int
            }
        )
df = pd.DataFrame(result)
df

,bf_size,set_size,fp_count,ones_count
0,8,5,2,3
1,8,50,42,8
2,8,500,492,8
3,8,5000,4992,8
4,8,5000000,4999992,8
5,64,5,1,4
6,64,50,13,37
7,64,500,436,64
8,64,5000,4936,64
9,64,5000000,4999936,64


# Задание 2

Реализуем Bloom Filter на к хеш-функций.

In [15]:
class numpyBloomFilter_k:
    def __init__(self, k, n):
        self.k = k
        self.n = n
        self.bit_array = np.zeros(self.n, dtype=bool)

    def put(self, s):
        for i in range(self.k): # в цикле проходимся по k хэш-функций
            hash_value = mmh3.hash(s, i) % self.n
            self.bit_array[hash_value] = True

    def get(self, s):
        return all(self.bit_array[mmh3.hash(s, i) % self.n] for i in range(self.k))

    def size(self):
        return np.sum(self.bit_array) / self.k # кол-во единиц в массиве / k

### Тестируем

In [16]:
bf_sizes = [8, 64, 1024, 65536, 16777216]
set_sizes = [5, 50, 500, 5000, 5000000]

In [17]:
result_k = []

for k in [1, 2, 3, 4]:
    for n in (bf_sizes):
        for set_size in (set_sizes):
            bf_int = numpyBloomFilter_k(n=n, k=k)
            fp_count = 0
            with open(f'{set_size}') as file:
                for line in file:
                    if bf_int.get(line):
                        fp_count += 1
                    bf_int.put(line)
                ones_count_int = bf_int.size()
            result_k.append(np.array([k, n, set_size, fp_count, ones_count_int]))

In [18]:
df_k = pd.DataFrame(result_k)
df_k

,0,1,2,3,4
0,1.0,8.0,5.0,2.0,3.00
1,1.0,8.0,50.0,42.0,8.00
2,1.0,8.0,500.0,492.0,8.00
3,1.0,8.0,5000.0,4992.0,8.00
4,1.0,8.0,5000000.0,4999992.0,8.00
...,...,...,...,...,...
95,4.0,16777216.0,5.0,0.0,5.00
96,4.0,16777216.0,50.0,0.0,50.00
97,4.0,16777216.0,500.0,0.0,500.00
98,4.0,16777216.0,5000.0,0.0,4995.50


# Задание 3

Реализуем Counting Bloom Filter на к хеш-функций.

In [19]:
class CountingBloomFilter:
    def __init__(self, k, n, cap=1):
        self.k = k
        self.n = n
        self.cap = cap # кол-во бит на каждый счетчик
        self.bit_array = np.zeros(self.n * cap, dtype=np.uint8)

    def put(self, s):
        for i in range(self.k): # для кажой хэш-функции увеличиваем счетчики
            hash_value = mmh3.hash(s, i) % self.n
            self.bit_array[hash_value] = min(self.bit_array[hash_value] + 1, (1 << self.cap) - 1)

    def get(self, s):
        return all(self.bit_array[mmh3.hash(s, i) % self.n] > 0 for i in range(self.k))

    def delete(self, s):
        for i in range(self.k): # для кажой хэш-функции уменьшаем счетчики
            hash_value = mmh3.hash(s, i) % self.n
            if self.bit_array[hash_value] > 0:
                self.bit_array[hash_value] -= 1

    def size(self):
        return np.sum(self.bit_array) / self.k # сумма счетчиков / k

### Тестируем

Посмотрим cap на высоком уровне, cap in [4,5]. А сочетания значений n и k протестируем. Почему так: высокий cap поможет избежать переполнения и позволяет нам взять побольше счетчиков




In [20]:
def exp(cap, k, n, set_size, exp_num):
    utils.gen_uniq_seq(f'cbf_{exp_num}.csv', 5000)
    cbf = CountingBloomFilter(cap=cap, k=k, n=n)
    fp_count = 0

    with open(f'cbf_{exp_num}.csv') as s:
        for i in s:
            if cbf.get(i):
                fp_count += 1
            cbf.put(i)
        ones_count = cbf.size()

    return cap, k, n, fp_count, ones_count

In [21]:
bf_sizes = [8, 64, 1024, 65536, 16777216]
set_sizes = [5, 50, 500, 5000, 5000000]
k_values = [1, 2, 3, 4]
cap_values = [4, 5]
results_cbf = []

exp_num = 0
for cap in cap_values:
  for n in bf_sizes:
      for ss in set_sizes:
          for k in k_values:
              results_cbf.append(exp(cap, k, n, ss, exp_num))
              exp_num += 1
df_cbf = pd.DataFrame(results_cbf)
df_cbf.columns = ['cap', 'k', 'n', 'fp_count', 'ones_count']

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [22]:
df_cbf

,cap,k,n,fp_count,ones_count
0,4,1,8,4992,120.0
1,4,2,8,4996,60.0
2,4,3,8,4994,40.0
3,4,4,8,4996,30.0
4,4,1,8,4992,120.0
...,...,...,...,...,...
195,5,4,16777216,0,5000.0
196,5,1,16777216,1,5000.0
197,5,2,16777216,0,5000.0
198,5,3,16777216,0,5000.0


# Задание 4

Реализуем HyperLogLog.


In [23]:
class HyperLogLog:
    def __init__(self, b):
        self.b = b
        self.m = 1 << b
        self.registers = [0] * self.m
        if self.m == 16:
            self.a = 0.673
        elif self.m == 32:
            self.a = 0.697
        elif self.m == 64:
            self.a = 0.709
        else:
            self.a = 0.7213 / (1 + 1.079 / self.m)

    def put(self, s):
        x = mmh3.hash(s, signed=False)
        x_bin = bin(x)[2:].zfill(32)
        j = int(x_bin[:self.b], 2)
        w = x_bin[self.b:]
        self.registers[j] = max(self.registers[j], len(w) - len(w.lstrip('0')) + 1)

    def est_size(self):
        Z = sum(2.0 ** -reg for reg in self.registers)
        E = self.a * self.m * self.m / Z
        if E <= 2.5 * self.m:
            V = self.registers.count(0)
            if V > 0:
                E = self.m * math.log(self.m / V)
        elif E > (1 / 30.0) * (1 << 32):
            E = -(1 << 32) * math.log(1 - E / (1 << 32))
        return E

### Тестируем

In [24]:
def hll_exp(pattern, true_size, b):
    utils.gen_grouped_seq('hll_exp.txt', pattern)
    hll = HyperLogLog(b)

    with open('hll_exp.txt', "r") as f:
        for line in f:
            key = line.strip().split(':')[0]
            hll.put(key)

    est_size = hll.est_size()
    rel_error = abs(est_size - true_size) / true_size

    print('true size:', true_size)
    print('est size:', est_size)
    print('error:, ', rel_error)
    print('\n')

In [25]:
hll_exp(pattern=[(1000, 1), (100, 1000)], true_size=1100, b=15)
hll_exp(pattern=[(50000, 1), (500, 5000)], true_size=50500, b=15)
hll_exp(pattern=[(1000000, 1), (1000, 10000)], true_size=1001000, b=15)

true size: 1100
est size: 1095.09674651748
error:,  0.004457503165927351


true size: 50500
est size: 50429.33822371157
error:,  0.0013992430948203379


true size: 1001000
est size: 1010607.3388392192
error:,  0.009597741098121063




In [26]:
## проверим разную точность

hll_exp(pattern=[(1000, 1), (100, 1000)], true_size=1100, b=10)
hll_exp(pattern=[(1000, 1), (100, 1000)], true_size=1100, b=15)
hll_exp(pattern=[(1000, 1), (100, 1000)], true_size=1100, b=20)

true size: 1100
est size: 1125.9794721955188
error:,  0.023617701995926218


true size: 1100
est size: 1095.09674651748
error:,  0.004457503165927351


true size: 1100
est size: 1099.576327126088
error:,  0.0003851571581018024


